In [41]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/02/27 15:22:07 WARN Utils: Your hostname, localhost.localdomain resolves to a loopback address: 127.0.0.1; using 10.0.2.20 instead (on interface enp0s3)
22/02/27 15:22:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/02/27 15:22:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/27 15:22:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
df = spark.read \
.option("header","true") \
.csv("../data/raw/fhvhv_tripdata_2021-02.csv")

In [10]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [11]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [22]:
schema = types.StructType([
	types.StructField('hvfhs_license_num',types.StringType(),True),
	types.StructField('dispatching_base_num',types.StringType(),True),
	types.StructField('pickup_datetime',types.TimestampType(),True),
	types.StructField('dropoff_datetime',types.TimestampType(),True),
	types.StructField('PULocationID',types.IntegerType(),True),
	types.StructField('DOLocationID',types.IntegerType(),True),
	types.StructField('SR_Flag',types.StringType(),True)
])

In [23]:
df = spark.read \
.option("header","true") \
.schema(schema) \
.csv("../data/raw/fhvhv_tripdata_2021-02.csv")

In [28]:
df = df.repartition(24)

In [29]:
df.write.parquet("../data/pq/",mode='overwrite')

In [55]:
df = spark.read.parquet("../data/pq/")

In [86]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [85]:
df.count()

11613942

In [32]:
df.registerTempTable("hvfhw_2021_02")

In [34]:
df.show(10)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02617|2021-02-02 23:37:38|2021-02-02 23:53:18|         152|         242|   null|
|           HV0003|              B02887|2021-02-02 09:56:51|2021-02-02 10:20:33|          28|           7|   null|
|           HV0003|              B02869|2021-02-05 12:58:01|2021-02-05 13:08:53|          47|         147|   null|
|           HV0003|              B02872|2021-02-05 07:32:17|2021-02-05 07:43:59|          50|         244|   null|
|           HV0003|              B02867|2021-02-02 02:27:45|2021-02-02 02:39:29|         188|          61|   null|
|           HV0003|              B02869|2021-02-03 09:09:52|2021-02-03 10:04:17|

In [40]:
spark.sql("""
select count(1) from hvfhw_2021_02
where pickup_datetime >= '2021-02-15 00:00:00' and pickup_datetime <= '2021-02-15 23:59:59'
""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+



In [58]:
timeformat="yyyy-MM-dd'T'HH:mm:ss.SSS"

In [79]:
df \
 .withColumn('pickup_date',F.to_date(df.pickup_datetime)) \
 .withColumn('dropoff_date',F.to_date(df.dropoff_datetime)) \
 .withColumn('duration',(F.unix_timestamp(df.dropoff_datetime, format=timeformat) - F.unix_timestamp(df.pickup_datetime, format=timeformat))) \
 .select('pickup_date','dropoff_date','duration') \
 .orderBy('duration',ascending=False) \
 .show()

+-----------+------------+--------+
|pickup_date|dropoff_date|duration|
+-----------+------------+--------+
| 2021-02-11|  2021-02-12|   75540|
| 2021-02-17|  2021-02-18|   57221|
| 2021-02-20|  2021-02-21|   44039|
| 2021-02-03|  2021-02-04|   40653|
| 2021-02-19|  2021-02-20|   37577|
| 2021-02-25|  2021-02-26|   35010|
| 2021-02-20|  2021-02-20|   34806|
| 2021-02-18|  2021-02-19|   34612|
| 2021-02-18|  2021-02-18|   34555|
| 2021-02-10|  2021-02-11|   34169|
| 2021-02-10|  2021-02-10|   32476|
| 2021-02-25|  2021-02-25|   32439|
| 2021-02-21|  2021-02-22|   32223|
| 2021-02-09|  2021-02-10|   32087|
| 2021-02-06|  2021-02-06|   31447|
| 2021-02-02|  2021-02-02|   30913|
| 2021-02-10|  2021-02-10|   30856|
| 2021-02-09|  2021-02-09|   30732|
| 2021-02-21|  2021-02-22|   30660|
| 2021-02-05|  2021-02-06|   30511|
+-----------+------------+--------+
only showing top 20 rows



In [83]:
df_most_frequent = spark.sql("""
select dispatching_base_num,
     COUNT(dispatching_base_num) AS TOTAL
     FROM hvfhw_2021_02
     GROUP BY dispatching_base_num
     ORDER by COUNT(dispatching_base_num) desc
""")

In [84]:
df_most_frequent.show()

+--------------------+-------+
|dispatching_base_num|  TOTAL|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
|              B02869| 429720|
|              B02887| 322331|
|              B02871| 312364|
|              B02864| 311603|
|              B02866| 311089|
|              B02878| 305185|
|              B02682| 303255|
|              B02617| 274510|
|              B02883| 251617|
|              B02884| 244963|
|              B02882| 232173|
|              B02876| 215693|
|              B02879| 210137|
|              B02867| 200530|
|              B02877| 198938|
+--------------------+-------+
only showing top 20 rows



In [89]:
df_zones = spark.read \
.option("header","true") \
.csv("../taxi+_zone_lookup.csv")

In [90]:
df_zones.write.parquet("../data/pq_zones",mode='overwrite')

In [92]:
df_zones = spark.read.parquet("../data/pq_zones")

In [93]:
df_zones.registerTempTable("ms_zones")

In [111]:
df_zone_most = spark.sql("""
SELECT CONCAT(pz.Zone,'/',dz.Zone) AS pickuplocation_and_droplocation,COUNT(CONCAT(pz.Zone,'/',dz.Zone)) AS total
FROM hvfhw_2021_02
INNER JOIN ms_zones AS pz on hvfhw_2021_02.PULocationID = pz.LocationID
INNER JOIN ms_zones AS dz on hvfhw_2021_02.DOLocationID = dz.LocationID
GROUP BY pz.Zone,dz.Zone
ORDER BY COUNT(CONCAT(pz.Zone,'/',dz.Zone)) DESC
""")

In [115]:
df_zone_most.write.parquet("../data/most_zone/",mode="overwrite")

In [116]:
df_zone_most = spark.read.parquet("../data/most_zone/")

In [119]:
df_zone_most.head(10)

[Row(pickuplocation_and_droplocation='Saint Michaels Cemetery/Woodside/Fordham South', total=1),
 Row(pickuplocation_and_droplocation='Lincoln Square East/Laurelton', total=1),
 Row(pickuplocation_and_droplocation='Manhattan Beach/Claremont/Bathgate', total=1),
 Row(pickuplocation_and_droplocation='Greenpoint/Country Club', total=1),
 Row(pickuplocation_and_droplocation='Grymes Hill/Clifton/Meatpacking/West Village West', total=1),
 Row(pickuplocation_and_droplocation='Willets Point/Bedford', total=1),
 Row(pickuplocation_and_droplocation='Inwood/Jamaica Estates', total=1),
 Row(pickuplocation_and_droplocation='Bensonhurst West/Westchester Village/Unionport', total=1),
 Row(pickuplocation_and_droplocation='Kew Gardens/Willets Point', total=1),
 Row(pickuplocation_and_droplocation='Flushing Meadows-Corona Park/Van Cortlandt Park', total=1)]